
# Python project - Hotel


# Importing data

In [3]:
import pandas as pd
import openpyxl
import numpy as np
from utils import satisfaction


In [4]:
guests= pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\guests.xlsx")
guests

,Unnamed: 0,guest,discount
0,0,guest_1,0.09
1,1,guest_2,0.00
2,2,guest_3,0.07
3,3,guest_4,0.00
4,4,guest_5,0.10
...,...,...,...
3995,3995,guest_3996,0.00
3996,3996,guest_3997,0.15
3997,3997,guest_3998,0.07
3998,3998,guest_3999,0.08


In [5]:
hotels = pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\hotels.xlsx")
hotels

,Unnamed: 0,hotel,rooms,price
0,0,hotel_1,13,273
1,1,hotel_2,18,92
2,2,hotel_3,12,141
3,3,hotel_4,18,157
4,4,hotel_5,7,298
...,...,...,...,...
395,395,hotel_396,5,212
396,396,hotel_397,12,68
397,397,hotel_398,14,102
398,398,hotel_399,16,284


In [7]:
preferences = pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\preferences.xlsx")
preferences

,Unnamed: 0,guest,hotel,priority
0,0,guest_1,hotel_168,1
1,1,guest_1,hotel_207,2
2,2,guest_1,hotel_222,3
3,3,guest_1,hotel_124,4
4,4,guest_1,hotel_223,5
...,...,...,...,...
99528,99528,guest_4000,hotel_123,28
99529,99529,guest_4000,hotel_396,29
99530,99530,guest_4000,hotel_161,30
99531,99531,guest_4000,hotel_330,31


# Random Allocation

In [8]:
class HotelAllocation:
    def __init__(self, hotels, guests):
        """
        Initialize the HotelAllocation.

        Parameters:
        - hotels (pd.DataFrame): DataFrame containing information about hotels.
        - guests (pd.DataFrame): DataFrame containing information about guests.
        """
        #we use copies to avoid modifying the original dataframes
        self.hotels = hotels.copy()
        self.guests = guests.copy()
    
    def accomodate_single_guest():
        pass

    def accomodate_guests(self):
        allocations = []

        for _, guest_row in self.guests.iterrows(): #iterrows returns a series for each row
            allocation_entry = self.accomodate_single_guest(guest_row) #allocate the current guest to a random avaiable hotel
            if allocation_entry is not None:
                allocations.append(allocation_entry)

        allocation_df = pd.DataFrame(allocations, columns=['guest_id', 'hotel_id', 'paid_price'])
        return allocation_df

class RandomHotelAllocation(HotelAllocation):
       """ 
       Allocate a guest to a random available hotel based on availability.

        Parameters:
        - guest_row (pd.Series): A row from the guests DataFrame.

        Returns:
        - dict: Allocation information with keys 'guest_id', 'hotel_id', and 'paid_price'.
       """
    def accomodate_single_guest(self, guest_row):
        available_hotels = self.hotels[self.hotels['rooms'] > 0]
        if available_hotels.empty:
            return None

        random_hotel_id = np.random.choice(available_hotels.index) #we randomly select a hotel with available rooms
        self.hotels.loc[random_hotel_id, 'rooms'] -= 1 #reduce the number of avaiable rooms in that hotel by 1
        
        #we calculate the discounted price that a guest will pay for staying in the randomly allocated hotel,
        #taking into account both the original price of the hotel and the guest's discount.
        price_paid = round(available_hotels.loc[random_hotel_id, 'price'] * (1 - guest_row['discount']), 2)
        satisfaction = calculate_satisfaction_percentage(guest_id, random_available_hotel_id, preferences)

        return {'guest_id': guest_row.name, 'hotel_id': random_hotel_id, 'paid_price': price_paid, 'satisfaction': satisfaction}


IndentationError: expected an indented block after function definition on line 2 (1455759318.py, line 3)

# Preference Allocation

In [ ]:
class PreferencesAllocator:
    def __init__(self, hotels, guests, preferences):
         """Initialize the AvailabilityBasedAllocator.

        Parameters:
        - hotels (pd.DataFrame): DataFrame containing information about hotels.
        - guests (pd.DataFrame): DataFrame containing information about guests.
        - preferences (pd.DataFrame): DataFrame containing guest preferences.
     """
        #we use copies to avoid modifying the original dataframes
        self.hotels = hotels.copy()
        self.guests = guests.copy()
        self.preferences = preferences.copy()
        
    def allocate_and_calculate(hotels, guests, preferences):
    """
    Allocate guests to their preferred hotels, calculate paid price and satisfaction.

    Parameters:
    - hotels (pd.DataFrame): DataFrame containing information about hotels.
    - guests (pd.DataFrame): DataFrame containing information about guests.
    - preferences (pd.DataFrame): DataFrame containing guest preferences.

    Returns:
    - list: List of allocation information for each guest, including guest ID, hotel ID,
            satisfaction, and paid price.
    """
    allocation_list = []

    for guest_id, guest_row in guests.iterrows():
        #the code is iterating through the guests in the guests DataFrame and, for each guest,
        #extracting the preferred hotels from the preferences DataFrame based on their ID.
        guest_preferred_hotels = preferences[preferences['guest'] == f'guest_{guest_id}']['hotel']

        for _, preferred_hotel_id in guest_preferred_hotels.items():
            hotel_index = int(preferred_hotel_id.lstrip('hotel_')) - 1
            preferred_hotel_row = hotels.loc[hotel_index]

            if preferred_hotel_row['rooms'] > 0:
                hotels.loc[hotel_index, 'rooms'] -= 1

                paid_price_coefficient = 1 - guest_row['discount']
                paid_price = preferred_hotel_row['price'] * paid_price_coefficient

                satisfaction = calculate_satisfaction_percentage(guest_id, preferred_hotel_id, preferences)

                allocation_entry = [guest_id, preferred_hotel_id, satisfaction, paid_price]
                allocation_list.append(allocation_entry)

                break

    return allocation_list

# Price allocation

In [ ]:
class PriceBasedAllocator:
    def __init__(self, hotels, guests, preferences):
        """
        Initialize the AvailabilityBasedAllocator.

        Parameters:
        - hotels (pd.DataFrame): DataFrame containing information about hotels.
        - guests (pd.DataFrame): DataFrame containing information about guests.
        - preferences (pd.DataFrame): DataFrame containing guest preferences.
        """
        #we use copies to avoid modifying the original dataframes 
        self.hotels = hotels
        self.guests = guests
        self.preferences = preferences

    def allocate_and_calculate(self):
        """
        Allocate guests to hotels based on price, calculate satisfaction and paid price.

        Returns:
        - pd.DataFrame: DataFrame containing allocation information.
        """
        allocation_list = []

        sorted_hotels = preferences.merge(hotels, on=['hotel']).merge(guests).sort_values(by='price')

        for group_key, group in sorted_hotels.groupby('hotel', sort=False):
            for id, row in group.iterrows():
                if group.iloc[0]['rooms'] == 0:
                    break
                group['rooms'] -= 1
                paid_price = self.calculate_paid_price(row)
                satisfaction = calculate_satisfaction_percentage(row['guest'], row['hotel'], preferences)
                allocation_entry = [row['guest'], row['hotel'], satisfaction, paid_price]
                allocation_list.append(allocation_entry)
        return pd.DataFrame(allocation_list, columns=['guest_id', 'hotel_id', 'satisfaction', 'paid_price'])
    def can_allocate_to_hotel(self, hotel_row, guest_id):
        """
        Check if a guest can be allocated to a hotel.

        Parameters:
        - hotel_row (pd.Series): Row of the hotel DataFrame.
        - guest_id: ID of the guest.

        Returns:
        - bool: True if allocation is possible, False otherwise.
        """
        return hotel_row['rooms'] > 0 and guest_id not in self.preferences.index

    def calculate_paid_price(self, row):
        return row['price'] * 1 - row['discount']

# Availability Allocation

In [ ]:
class AvailabilityBasedAllocator:
    def __init__(self, hotels, guests, preferences):
        """
        Initialize the AvailabilityBasedAllocator.

        Parameters:
        - hotels (pd.DataFrame): DataFrame containing information about hotels.
        - guests (pd.DataFrame): DataFrame containing information about guests.
        - preferences (pd.DataFrame): DataFrame containing guest preferences.
        """
        # Use copies to avoid modifying the original DataFrames
        self.hotels = hotels.copy()
        self.guests = guests.copy()
        self.preferences = preferences.copy()

    def allocate_and_calculate(self):
        """
        Allocate guests to hotels based on available rooms, calculate satisfaction, and paid price.

        Returns:
        - pd.DataFrame: DataFrame containing allocation information.
        """
        allocation_list = []

        # Sort hotels based on available rooms (desc order)
        sorted_hotels = self.hotels.sort_values(by='rooms', ascending=False)

        for _,hotel_row in sorted_hotels.iterrows():
            allocated_guests = set()  #iniziales an empty set and track guests already allocated to the current hotel
            for guest_row in self.guests.iterrows():
                guest_id = guest_row['guest_id']

                # Check if the guest can be allocated to the current hotel
                if can_allocate_to_hotel(hotel_row, guest_id, self.preferences) and guest_id not in allocated_guests:
                    paid_price = hotel_row['price'] * (1 - guest_row['discount'])
                    satisfaction = calculate_satisfaction_percentage(guest_id, hotel_row.name, self.preferences)
                    allocation_entry = [guest_id, hotel_row.name, satisfaction, paid_price]
                    allocation_list.append(allocation_entry)

                    # Update the set of allocated guests for the current hotel
                    allocated_guests.add(guest_id)

        return pd.DataFrame(allocation_list, columns=['guest_id', 'hotel_id', 'satisfaction', 'paid_price'])

# Output